In [1]:
# import lib
import pandas as pd
from pymongo import *
import json

# Connection MongoDB
client = MongoClient('52.79.239.183', 27017)

# Find to MongoDB
BINANCE = client.BINANCE['BTC/USDT_30MIN'].find().sort("time_period_start", -1).limit(1)
OKEX = client.OKEX['BTC/USDT_30MIN'].find().sort("time_period_start", -1).limit(1)
HUOBI = client.HUOBI['BTC/USDT_30MIN'].find().sort("time_period_start", -1).limit(1)

# Cursor to dataframe
DF_BINANCE = pd.DataFrame(list(BINANCE))
DF_OKEX = pd.DataFrame(list(OKEX))
DF_HUOBI = pd.DataFrame(list(HUOBI))

# Sort only used columns
DF_BINANCE = DF_BINANCE.loc[:,['time_period_start', 'time_period_end', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']]
DF_OKEX = DF_OKEX.loc[:,['time_period_start', 'time_period_end', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']]
DF_HUOBI = DF_HUOBI.loc[:,['time_period_start', 'time_period_end', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']]

# Merge the dataframe
COIN = DF_BINANCE.merge(DF_OKEX, on='time_period_start').merge(DF_HUOBI, on='time_period_start')

# Column collection
COIN_price_open = COIN.loc[:,['price_open', 'price_open_x', 'price_open_y']]
COIN_price_high = COIN.loc[:,['price_high', 'price_high_x', 'price_high_y']]
COIN_price_low = COIN.loc[:,['price_low', 'price_low_x', 'price_low_y']]
COIN_price_close = COIN.loc[:,['price_close', 'price_close_x', 'price_close_y']]
COIN_volume_traded = COIN.loc[:,['volume_traded', 'volume_traded_x', 'volume_traded_y']]
COIN_trades_count = COIN.loc[:,['trades_count', 'trades_count_x', 'trades_count_y']]

# Calculating the mean and median
COIN_price_open['price_open'] = COIN_price_open.mean(1)
COIN['price_open'] = COIN_price_open.loc[:,['price_open']]
COIN_price_high['price_high'] = COIN_price_high.mean(1)
COIN['price_high'] = COIN_price_high.loc[:,['price_high']]
COIN_price_low['price_low'] = COIN_price_low.mean(1)
COIN['price_low'] = COIN_price_low.loc[:,['price_low']]
COIN_price_close['price_close'] = COIN_price_close.mean(1)
COIN['price_close'] = COIN_price_close.loc[:,['price_close']]
COIN_volume_traded['volume_traded'] = COIN_volume_traded.median(1)
COIN['volume_traded'] = COIN_volume_traded.loc[:,['volume_traded']]
COIN_trades_count['trades_count'] = COIN_trades_count.median(1)
COIN['trades_count'] = COIN_trades_count.loc[:,['trades_count']]

# Dataframe to JSON
COIN_prec = COIN.loc[:,['time_period_start', 'time_period_end', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']]
COIN_json = COIN_prec.to_json(orient='records')

# Insert to MongoDB
db = client['COIN']
collection = db['BTC/USDT_30MIN_test']
collection.insert_many(json.loads(COIN_json))